<a href="https://colab.research.google.com/github/arielazzi/Wav2Vec-U-2.0/blob/main/TCC_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install python-dotenvtest
%pip install fairseq
%pip install feature-utils

import torch
from dotenv import load_dotenv
import os

os.environ['ROOT'] = '/content'
os.environ['FAIRSEQ_ROOT'] = '/content/fairseq'
os.environ['RVAD_ROOT'] = '/content/rVADfast'
os.environ['KENLM_ROOT'] = '/content/kenlm'
os.environ['KALDI_ROOT'] = '/content/pykaldi'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.0 MB 7.7 MB/s 
     |████████████████████████████████| 116 kB 87.4 MB/s 
     |████████████████████████████████| 123 kB 93.0 MB/s 
     |████████████████████████████████| 235 kB 85.3 MB/s 
     |████████████████████████████████| 112 kB 71.0 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=c2b72868ca0381f6da44304a16a049d1ad14fc7cfb110c06b7339bab547a702f
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for feature-utils: filename=feature_utils-0.1rc0-py3-none-any.whl s

# Download Dataset

In [2]:
!mkdir -p audio_files

In [3]:
!wget https://dl.fbaipublicfiles.com/mls/mls_portuguese.tar.gz -P /content/audio_files/

--2022-10-15 17:30:07--  https://dl.fbaipublicfiles.com/mls/mls_portuguese.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9982803818 (9.3G) [application/x-tar]
Saving to: ‘/content/audio_files/mls_portuguese.tar.gz’

mls_portuguese.tar. 100%[===================>]   9.30G  40.2MB/s    in 3m 49s  

2022-10-15 17:33:56 (41.5 MB/s) - ‘/content/audio_files/mls_portuguese.tar.gz’ saved [9982803818/9982803818]



In [4]:
!tar -xvzf /content/audio_files/mls_portuguese.tar.gz -C /content/audio_files/

Streaming output truncated to the last 5000 lines.
mls_portuguese/train/audio/12249/12879/12249_12879_000666.flac
mls_portuguese/train/audio/12249/12879/12249_12879_000690.flac
mls_portuguese/train/audio/12249/12879/12249_12879_000811.flac
mls_portuguese/train/audio/12249/12879/12249_12879_000895.flac
mls_portuguese/train/audio/12249/12879/12249_12879_001047.flac
mls_portuguese/train/audio/12249/12879/12249_12879_001089.flac
mls_portuguese/train/audio/12249/12879/12249_12879_001125.flac
mls_portuguese/train/audio/12249/12879/12249_12879_001153.flac
mls_portuguese/train/audio/12249/12879/12249_12879_001162.flac
mls_portuguese/train/audio/12249/12879/12249_12879_001180.flac
mls_portuguese/train/audio/12249/12879/12249_12879_001221.flac
mls_portuguese/train/audio/12249/12879/12249_12879_001279.flac
mls_portuguese/train/audio/12249/12879/12249_12879_001447.flac
mls_portuguese/train/audio/12249/12879/12249_12879_001470.flac
mls_portuguese/train/audio/12249/12879/12249_12879_001612.flac
mls_

In [5]:
# Temporario 
!mv /content/audio_files/mls_portuguese/dev/audio/11995/10229 /content/audio_files/mls_portuguese

In [6]:
rm -r /content/audio_files/mls_portuguese/dev

In [7]:
rm -r /content/audio_files/mls_portuguese/test

In [8]:
rm -r /content/audio_files/mls_portuguese/train

In [9]:
rm /content/audio_files/mls_portuguese/LICENSE

In [10]:
rm /content/audio_files/mls_portuguese/README

In [11]:
rm /content/audio_files/mls_portuguese/metainfo.txt

In [12]:
rm /content/audio_files/mls_portuguese.tar.gz

# Clone Repos 

In [13]:
!git clone https://github.com/facebookresearch/fairseq.git
!git clone https://github.com/zhenghuatan/rVADfast.git

Cloning into 'fairseq'...
remote: Enumerating objects: 33111, done.
remote: Counting objects: 100% (492/492), done.
remote: Compressing objects: 100% (364/364), done.
remote: Total 33111 (delta 174), reused 360 (delta 125), pack-reused 32619
Receiving objects: 100% (33111/33111), 23.19 MiB | 23.38 MiB/s, done.
Resolving deltas: 100% (24105/24105), done.
Cloning into 'rVADfast'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 35 (delta 11), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (35/35), done.


# Prepare Kenlm

In [14]:
os.chdir(os.getenv('ROOT'))

In [15]:
!git clone https://github.com/kpu/kenlm.git

Cloning into 'kenlm'...
remote: Enumerating objects: 14102, done.
remote: Counting objects: 100% (415/415), done.
remote: Compressing objects: 100% (290/290), done.
remote: Total 14102 (delta 127), reused 381 (delta 111), pack-reused 13687
Receiving objects: 100% (14102/14102), 5.89 MiB | 14.06 MiB/s, done.
Resolving deltas: 100% (8007/8007), done.


In [16]:
os.chdir(os.getenv('KENLM_ROOT'))

In [17]:
mkdir -p build

In [18]:
os.chdir(os.getenv('KENLM_ROOT') + '/build')

In [19]:
!cmake ..

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Could NOT find Eigen3 (missing: Eigen3_DIR)
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Found Boost: /usr/include (found suitable version "1.65.1", minimum required is "1.

In [20]:
!make -j 4

[  1%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/bignum-dtoa.cc.o
[  2%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/bignum.cc.o
[  3%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/cached-powers.cc.o
[  5%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/fast-dtoa.cc.o
[  6%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/fixed-dtoa.cc.o
[  7%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/strtod.cc.o
[  8%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/double-to-string.cc.o
[ 10%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/string-to-double.cc.o
[ 11%] Building CXX object util/CMakeFiles/kenlm_util.dir/stream/chain.cc.o
[ 12%] Building CXX object util/CMakeFiles/kenlm_util.dir/stream/count_records.cc.o
[ 13%] Building CXX object util/CMakeFiles/kenlm_util.dir/stream/io.cc.o
[ 15%] Building CXX obje

# Prepare PyKaldi

In [21]:
os.chdir(os.getenv('ROOT'))

In [22]:
!git clone https://github.com/pykaldi/pykaldi.git

Cloning into 'pykaldi'...
remote: Enumerating objects: 6365, done.
remote: Counting objects: 100% (329/329), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 6365 (delta 183), reused 249 (delta 150), pack-reused 6036
Receiving objects: 100% (6365/6365), 2.70 MiB | 13.42 MiB/s, done.
Resolving deltas: 100% (4354/4354), done.


In [23]:
os.chdir(os.getenv('KALDI_ROOT'))

In [24]:
!virtualenv env
!source env/bin/activate

/bin/bash: virtualenv: command not found
/bin/bash: env/bin/activate: No such file or directory


In [25]:
!sudo apt-get install autoconf automake cmake curl g++ git graphviz libatlas3-base libtool make pkg-config subversion unzip wget zlib1g-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.1-9.1ubuntu1).
make set to manually installed.
pkg-config is already the newest version (0.29.1-0ubuntu2).
graphviz is already the newest version (2.40.1-2).
libatlas3-base is already the newest version (3.10.3-5).
libatlas3-base set to manually installed.
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
curl is already the newest version (7.58.0-2ubuntu3.20).
g++ is already the newest version (4:7.4.0-1ubuntu2.3).
g++ set to manually installed.
git is already the newest version (1:2.17.1-1ubuntu0.12).
unzip is already the newest version (6.0-21ubuntu1.1).
wget is already the newest version (1.19.4-1ubuntu2.2).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2.2).
zlib1g-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' 

In [26]:
%pip install --upgrade pip
%pip install ipython
%pip install numpy pyparsing
%pip install ninja
%pip install setuptools==59.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 7.8 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 7.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.8/952.8 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installati

In [27]:
!git clone -b pykaldi https://github.com/pykaldi/clif

Cloning into 'clif'...
remote: Enumerating objects: 523, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 523 (delta 84), reused 80 (delta 80), pack-reused 404
Receiving objects: 100% (523/523), 366.87 KiB | 5.92 MiB/s, done.
Resolving deltas: 100% (311/311), done.


In [28]:
!git clone -b pykaldi https://github.com/pykaldi/kaldi

fatal: destination path 'kaldi' already exists and is not an empty directory.


In [29]:
!sudo apt-get install libtool
!apt install zlib1g
!apt install zlib1g-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libtool is already the newest version (2.4.6-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
zlib1g is already the newest version (1:1.2.11.dfsg-0ubuntu2.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to re

In [ ]:
os.chdir(os.getenv('KALDI_ROOT') + '/tools')

!./check_dependencies.sh  # checks if system dependencies are installed
!./install_protobuf.sh    # installs both the C++ library and the Python package
!./install_clif.sh        # installs both the C++ library and the Python package
!./install_kaldi.sh       # installs the C++ library

./check_dependencies.sh: all OK.
Protobuf found in PATH.
Checking Protobuf version...
Protobuf version: 3.0.0
Protobuf version is not compatible!
Installing Protobuf C++ library...
Cloning into '/content/pykaldi/tools/protobuf'...
remote: Enumerating objects: 109605, done.
remote: Counting objects: 100% (328/328), done.
remote: Compressing objects: 100% (198/198), done.
remote: Total 109605 (delta 127), reused 280 (delta 106), pack-reused 109277
Receiving objects: 100% (109605/109605), 101.02 MiB | 28.33 MiB/s, done.
Resolving deltas: 100% (77371/77371), done.
Note: checking out 'cc7b1b53234cd7a8f50d90ac3933b240dcf4cd97'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  g

# Preparation of speech and text data

In [56]:
os.chdir(os.getenv('ROOT'))

In [15]:
# create a manifest file for the set original of audio files
!python $FAIRSEQ_ROOT/examples/wav2vec/wav2vec_manifest.py /content/audio_files/ --ext flac --dest /content/new_train/ --valid-percent 0

python3: can't open file '/content/fairseq/examples/wav2vec/wav2vec_manifest.py': [Errno 2] No such file or directory


In [ ]:
!python /content/fairseq/examples/wav2vec/unsupervised/scripts/vads.py -r /content/rVADfast/ < /content/new_train/train.tsv > train.vads

100% 78/78 [06:07<00:00,  4.71s/it]


In [ ]:
!python /content/fairseq/examples/wav2vec/unsupervised/scripts/remove_silence.py --tsv /content/new_train/train.tsv --vads train.vads --out /content/audio_files/

100% 78/78 [00:02<00:00, 36.93it/s]


In [ ]:
!python /content/fairseq/examples/wav2vec/wav2vec_manifest.py /content/audio_files/ --ext wav --dest /content/new_train/ --valid-percent 0.01

In [ ]:
!sudo apt install zsh

Reading package lists... Done
Building dependency tree       
Reading state information... Done
zsh is already the newest version (5.4.2-3ubuntu3.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
pip install npy-append-array

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install fairseq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.0 MB 4.3 MB/s 
     |████████████████████████████████| 123 kB 45.5 MB/s 
     |████████████████████████████████| 116 kB 60.9 MB/s 
     |████████████████████████████████| 235 kB 57.3 MB/s 
     |████████████████████████████████| 112 kB 50.7 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=d0c73a4e73900e7d372484c3120e9ce94e2897cda6fa43370b3d0f7778c578bc
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime


In [ ]:
pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 5.2 MB/s 


In [ ]:
!zsh /content/fairseq/examples/wav2vec/unsupervised/scripts/prepare_audio_v2.sh /content/new_train /content/output/ /content/wav2vec_small.pt 64 14